In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd
# Crear capa de puntos a partir de las coordenadas ETRS89
# Crear geometría de puntos
df = pd.read_excel(r"E:\SAR_UVa\Coord_IFN_Extremadura.xlsx")
geometry = [Point(xy) for xy in zip(df['XETRS89_H30'], df['YETRS89_H30'])]

# Crear GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:25830')

print("Capa de puntos creada exitosamente")
print(f"Número de puntos: {len(gdf)}")
print(f"CRS: {gdf.crs}")
print(f"\nPrimeros 5 puntos:")
print(gdf.head())
gdf.to_file(r"E:\SAR_UVa\shps\puntos_ifn_rioja.geojson")



In [ ]:
df_coh = pd.read_csv(r"E:\SAR_UVa\CSV'S\valores_coherencia_extremadura_modificado.csv")
print(df_coh.head())

In [ ]:
import rasterio
import rioxarray as rxr

input_path = r"E:\SAR_UVa\coherence_stack.nc"

# Método 1: Ver con rioxarray
raster = rxr.open_rasterio(input_path)
print("Coordenadas:", raster.coords)
print("Atributos:", raster.attrs)

# Método 2: Ver con rasterio
with rasterio.open(input_path) as src:
    print("\nNúmero de bandas:", src.count)
    print("\nDescripciones de bandas:")
    for i in range(min(5, src.count)):  # Primeras 5 bandas
        print(f"  Banda {i+1}: {src.descriptions[i]}")
    
    print("\nTags de la primera banda:")
    print(src.tags(1))
    
    print("\nMetadata global:")
    print(src.meta)

In [ ]:
# Plot de serie temporal por ID
# Ajusta el ID y, si aplica, la banda
id_parcela = 23
band_seleccionada = None  # ej.: 'VV' o 'VH'

# Asegurar fecha en formato datetime
if 'fecha' in df_coh.columns:
    df_coh['fecha'] = pd.to_datetime(df_coh['fecha'])

filtro = df_coh['ID_PARCELA'] == id_parcela
if band_seleccionada is not None and 'band' in df_coh.columns:
    filtro &= df_coh['band'] == band_seleccionada

serie = (
    df_coh.loc[filtro]
    .sort_values('fecha')
    .set_index('fecha')['coherencia']
)

import matplotlib.dates as mdates

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(serie.index, serie.values, linewidth=1)
ax.set_title(f"Serie temporal de coherencia - ID {id_parcela}" + (f" - {band_seleccionada}" if band_seleccionada else ""))
ax.set_xlabel("Fecha")
ax.set_ylabel("Coherencia")
ax.grid(True, alpha=0.3)

# Ticks mensuales
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

## Codigo de calculo de parametros estadisticos


In [ ]:
"""
Script para calcular estadísticas de series temporales con AUC del año medio
Incluye:
- Área bajo la curva (AUC) del año medio por banda
- Media y desviación estándar por estación
- ACF en lags específicos
- Q-test (Ljung-Box)
- Índices del periodograma
"""

import numpy as np
import pandas as pd
from scipy.signal import welch
from scipy import integrate
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import acf
from pathlib import Path

# ============================================================================
# CONFIGURACIÓN
# ============================================================================

# Ruta del archivo CSV de entrada
input_file = r"E:\SAR_UVa\CSV'S\valores_coherencia_extremadura_modificado.csv"
output_file = 'estadisticas_series_temporales_.csv'

# Parámetros de análisis
fs = 1.0      # 1 observación por semana
nperseg = 52  # ventana anual
freq_muestreo = 52  # frecuencia de muestreo anual (semanas)

# Lags para ACF
acf_lags = [26, 52, 78, 104, 156]

# Lags para Q-test (Ljung-Box)
qtest_lags = [1, 2, 3, 26, 52, 104, 156]

# ============================================================================
# CARGA DE DATOS
# ============================================================================

print("=" * 80)
print("ANÁLISIS DE SERIES TEMPORALES CON AUC")
print("=" * 80)

if not Path(input_file).exists():
    print(f"❌ Error: El archivo {input_file} no existe")
    exit(1)

band_weekly = pd.read_csv(input_file)
band_weekly['fecha'] = pd.to_datetime(band_weekly['fecha'])

print(f"\n✅ Datos cargados desde: {input_file}")
print(f"   Registros: {len(band_weekly)}")
print(f"   Columnas: {band_weekly.columns.tolist()}")

# Obtener bandas disponibles
bands = band_weekly['band'].unique()
print(f"   Bandas encontradas: {bands}")

# ============================================================================
# PROCESAMIENTO PRINCIPAL
# ============================================================================

results = []
total_parcelas = len(band_weekly['ID_PARCELA'].unique())
contador = 0

for fid in band_weekly['ID_PARCELA'].unique():
    contador += 1
    if contador % 50 == 0:
        print(f"Procesando parcela {contador}/{total_parcelas}...")
    
    # Obtener la especie principal para este FID
    sp_ppal = band_weekly[band_weekly['ID_PARCELA'] == fid]['SP_PPAL'].iloc[0]
    
    # Iterar por cada banda
    for band in bands:
        
        # Filtrar datos por FID y banda
        data_fid_band = band_weekly[
            (band_weekly['ID_PARCELA'] == fid) &
            (band_weekly['band'] == band)
        ].copy()
        
        if len(data_fid_band) == 0:
            continue
        
        # Serie temporal completa
        ts = (
            data_fid_band
            .groupby('fecha')['coherencia']
            .mean()
            .dropna()
            .sort_index()
        )
        
        if len(ts) < nperseg:
            continue
        
        # Diccionario para almacenar resultados de este FID-banda
        row_data = {
            'ID_PARCELA': fid,
            'SP_PPAL': sp_ppal,
            'band': band
        }
        
        # ========== 0. ÁREA BAJO LA CURVA DEL AÑO MEDIO POR BANDA ==========
        # Obtener el año medio (agregación semanal por semana del año)
        data_fid_band['week_of_year'] = data_fid_band['fecha'].dt.isocalendar().week
        
        # Agrupar por semana del año y calcular la media
        yearly_mean = (
            data_fid_band
            .groupby('week_of_year')['coherencia']
            .mean()
            .sort_index()
        )
        
        if len(yearly_mean) > 0:
            # Calcular AUC usando integración trapezoidal
            x = np.arange(len(yearly_mean))  # semanas 0-51
            y = yearly_mean.values
            
            # AUC con método trapezoidal
            auc = integrate.trapezoid(y, x)
            row_data['auc_yearly_mean'] = auc
            
            # AUC normalizado (dividido por número de semanas)
            row_data['auc_yearly_mean_normalized'] = auc / len(yearly_mean)
            
            # Valor promedio del año (alternativa simple)
            row_data['mean_yearly_value'] = np.mean(y)
            
            # Mínimo y máximo del año medio
            row_data['min_yearly_value'] = np.min(y)
            row_data['max_yearly_value'] = np.max(y)
            row_data['std_yearly_value'] = np.std(y)
        else:
            row_data['auc_yearly_mean'] = np.nan
            row_data['auc_yearly_mean_normalized'] = np.nan
            row_data['mean_yearly_value'] = np.nan
            row_data['min_yearly_value'] = np.nan
            row_data['max_yearly_value'] = np.nan
            row_data['std_yearly_value'] = np.nan
        
        # ========== 1. MEDIA Y DESVIACIÓN ESTÁNDAR POR ESTACIÓN ==========
        data_fid_band['season'] = data_fid_band['fecha'].dt.quarter.map({
            1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'
        })
        
        for season in ['Winter', 'Spring', 'Summer', 'Fall']:
            season_data = data_fid_band[data_fid_band['season'] == season]['coherencia']
            row_data[f'mean_{season}'] = season_data.mean() if len(season_data) > 0 else np.nan
            row_data[f'std_{season}'] = season_data.std() if len(season_data) > 0 else np.nan
        
        # ========== 2. ACF EN LAGS ESPECÍFICOS ==========
        try:
            # Calcular ACF
            acf_values = acf(ts.values, nlags=max(acf_lags), fft=True)
            
            for lag in acf_lags:
                if lag < len(acf_values):
                    row_data[f'acf_lag_{lag}'] = acf_values[lag]
                else:
                    row_data[f'acf_lag_{lag}'] = np.nan
        except Exception as e:
            print(f"⚠️  Error ACF para ID_PARCELA={fid}, band={band}: {e}")
            for lag in acf_lags:
                row_data[f'acf_lag_{lag}'] = np.nan
        
        # ========== 3. Q-TEST (LJUNG-BOX) EN LAGS ESPECÍFICOS ==========
        try:
            # Calcular Ljung-Box test
            lb_result = acorr_ljungbox(ts.values, lags=qtest_lags, return_df=True)
            
            for lag in qtest_lags:
                if lag in lb_result.index:
                    row_data[f'qtest_stat_{lag}'] = lb_result.loc[lag, 'lb_stat']
                    row_data[f'qtest_pvalue_{lag}'] = lb_result.loc[lag, 'lb_pvalue']
                else:
                    row_data[f'qtest_stat_{lag}'] = np.nan
                    row_data[f'qtest_pvalue_{lag}'] = np.nan
        except Exception as e:
            print(f"⚠️  Error Q-test para ID_PARCELA={fid}, band={band}: {e}")
            for lag in qtest_lags:
                row_data[f'qtest_stat_{lag}'] = np.nan
                row_data[f'qtest_pvalue_{lag}'] = np.nan
        
        # ========== 4. ÍNDICES DEL PERIODOGRAMA ==========
        try:
            # Calcular periodograma con Welch
            freqs, power = welch(
                ts.values, 
                fs=fs, 
                detrend='constant',
                scaling='density'
            )
            
            # Convertir frecuencias a periodos
            mask = freqs > 0
            periodos = 1.0 / freqs[mask]
            power = power[mask]
            
            # Ciclos de interés
            ciclos = np.array([freq_muestreo/3, freq_muestreo/2, freq_muestreo])
            
            # Encontrar posiciones de los ciclos
            pos_ciclos = [np.argmin(np.abs(periodos - ciclo)) for ciclo in ciclos]
            
            # Extraer bandas de los ciclos
            bands_power = power[pos_ciclos]
            
            # 4.1 Seasonality Mode (periodo con máxima potencia)
            max_idx = np.argmax(bands_power)
            row_data['seasonality_mode'] = ciclos[max_idx]
            
            # 4.2 Fisher Kappa (max / mean)
            max_power = np.max(bands_power)
            mean_power = np.mean(bands_power)
            row_data['fisher_kappa'] = max_power / mean_power if mean_power > 0 else np.nan
            
            # 4.3 Seasonality Stability
            power_ciclo_anual_adelante = np.sum(power[pos_ciclos[2]:])
            row_data['seasonality_stability'] = (
                max_power / power_ciclo_anual_adelante 
                if power_ciclo_anual_adelante > 0 else np.nan
            )
            
            # 4.4 Plurianual Cycles
            power_plurianual = np.sum(power[:pos_ciclos[2]])
            power_total = np.sum(power)
            row_data['plurianual_cycles'] = (
                power_plurianual / power_total 
                if power_total > 0 else np.nan
            )
            
            # 4.5 Seasonality Amplitude
            row_data['seasonality_amplitude'] = max_power
            
        except Exception as e:
            print(f"⚠️  Error Periodograma para fid={fid}, band={band}: {e}")
            row_data['seasonality_mode'] = np.nan
            row_data['fisher_kappa'] = np.nan
            row_data['seasonality_stability'] = np.nan
            row_data['plurianual_cycles'] = np.nan
            row_data['seasonality_amplitude'] = np.nan
        
        # Agregar fila a resultados
        results.append(row_data)

# ============================================================================
# GUARDAR RESULTADOS
# ============================================================================

# Crear DataFrame con resultados
df_results = pd.DataFrame(results)

# Reordenar columnas
cols = ['ID_PARCELA', 'SP_PPAL', 'band'] + [col for col in df_results.columns if col not in ['ID_PARCELA', 'SP_PPAL', 'band']]
df_results = df_results[cols]

# Guardar CSV
df_results.to_csv(output_file, index=False)

print("\n" + "=" * 80)
print("✅ ANÁLISIS COMPLETADO")
print("=" * 80)
print(f"Archivo guardado en: {output_file}")
print(f"Total de registros: {len(df_results)}")
print(f"\nColumnas generadas ({len(df_results.columns)}):")
for i, col in enumerate(df_results.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\n📊 Resumen de AUC por banda:")
auc_summary = df_results.groupby('band')[['auc_yearly_mean', 'auc_yearly_mean_normalized', 'mean_yearly_value']].describe()
print(auc_summary)

print(f"\nPrimeras 5 filas:")
print(df_results.head())


In [1]:
4790000/13927095


0.34393389289008225

In [ ]:
import pandas as pd
import numpy as np

# Leer el CSV
df_coh = pd.read_csv(r"E:\SAR_UVa\CSV'S\valores_coherencia_extremadura.csv")

print("Columnas originales:", df_coh.columns.tolist())
print("Forma:", df_coh.shape)

# Identificar columnas de variables (todas excepto las de ID)
id_cols = ['Unnamed: 0', 'ID', 'ID_PARCELA']
var_cols = [col for col in df_coh.columns if col not in id_cols]

print(f"\nNúmero de columnas de variables: {len(var_cols)}")

# Generar rango de fechas
# Asumiendo frecuencia semanal desde una fecha inicial
fecha_inicio = pd.Timestamp('2017-03-01')
fechas = pd.date_range(start=fecha_inicio, periods=len(var_cols), freq='6D')  # Semanal
# Si es diario, cambiar 'W' por 'D'
# Si es mensual, cambiar 'W' por 'MS'

# Renombrar columnas
rename_dict = {col: fecha.strftime('%Y-%m-%d') for col, fecha in zip(var_cols, fechas)}
df_coh_renamed = df_coh.rename(columns=rename_dict)

# Mantener solo ID_PARCELA y las columnas de fechas (eliminar Unnamed y ID si prefieres)
df_coh_renamed = df_coh_renamed[['ID_PARCELA'] + list(rename_dict.values())]

print("\nPrimeras columnas renombradas:", df_coh_renamed.columns[:5].tolist())
print(df_coh_renamed.head())

# Guardar
df_coh_renamed.to_csv(r"E:\SAR_UVa\CSV'S\valores_coherencia_extremadura_con_fechas.csv", index=False)
print("\n✅ Archivo guardado: valores_coherencia_extremadura_con_fechas.csv")


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import MSTL
import warnings
warnings.filterwarnings('ignore')

# Cargar el archivo con fechas
df_coh_renamed = pd.read_csv(r"E:\SAR_UVa\CSV'S\valores_coherencia_extremadura_con_fechas.csv")

print("Forma del df original:", df_coh_renamed.shape)
print("Primeras columnas:", df_coh_renamed.columns[:5].tolist())

# Separar ID_PARCELA de las fechas
id_col = df_coh_renamed['ID_PARCELA']
fecha_cols = [col for col in df_coh_renamed.columns if col != 'ID_PARCELA']

# Convertir fechas a datetime
fechas = pd.to_datetime(fecha_cols)

# ========== PARÁMETROS MSTL (MÚLTIPLES ESTACIONALIDADES) ==========
seasonal_periods = [52]  # semanas (aprox. 6 meses, 1 año, 2 años)
# ================================================================

resultados_mstl = []
total_ids = len(df_coh_renamed)
min_required = 2 * max(seasonal_periods) + 1

for idx, row in df_coh_renamed.iterrows():
    if (idx + 1) % 50 == 0:
        print(f"Procesando {idx + 1}/{total_ids}...")
    
    id_parcela = row['ID_PARCELA']
    valores = row[fecha_cols].values
    ts = pd.Series(valores, index=fechas)
    
    # Remuestrear semanalmente
    ts_resampleado = ts.resample('W').mean()
    
    # MSTL no admite NaN: eliminar faltantes
    ts_resampleado = ts_resampleado.dropna()
    
    if len(ts_resampleado) > min_required:
        try:
            mstl = MSTL(ts_resampleado, periods=seasonal_periods)
            result = mstl.fit()
            
            trend = result.trend.values
            residual = result.resid.values
            seasonal_df = result.seasonal
            original = ts_resampleado.values
            fechas_resampleadas = ts_resampleado.index
            
            # Asegurar nombres consistentes para estacionales
            if isinstance(seasonal_df, pd.Series):
                seasonal_df = seasonal_df.to_frame()
            if len(seasonal_df.columns) == len(seasonal_periods):
                seasonal_df = seasonal_df.copy()
                seasonal_df.columns = [f"seasonal_{p}" for p in seasonal_periods]
            
            for i, fecha in enumerate(fechas_resampleadas):
                row_out = {
                    'ID_PARCELA': id_parcela,
                    'fecha': fecha,
                    'original': original[i],
                    'trend': trend[i],
                    'residual': residual[i]
                }
                for p in seasonal_periods:
                    col_name = f"seasonal_{p}"
                    if col_name in seasonal_df.columns:
                        row_out[col_name] = seasonal_df.iloc[i][col_name]
                    else:
                        # Fallback por índice si los nombres no coinciden
                        col_idx = seasonal_periods.index(p)
                        row_out[col_name] = seasonal_df.iloc[i].iloc[col_idx]
                resultados_mstl.append(row_out)
        except Exception as e:
            print(f"⚠️  Error MSTL para ID {id_parcela}: {e}")
    else:
        print(f"⚠️  ID {id_parcela}: insuficientes datos ({len(ts_resampleado)} puntos, mínimo requerido: {min_required})")

df_mstl = pd.DataFrame(resultados_mstl)

print("\n✅ Descomposición MSTL completada")
print(f"Períodos estacionales utilizados: {seasonal_periods}")
print(f"Total de registros: {len(df_mstl)}")
print("\nEstructura del resultado:")
print(df_mstl.head(10))
print("\nEstadísticas:")
print(df_mstl[['original', 'trend', 'residual'] + [f'seasonal_{p}' for p in seasonal_periods]].describe())

# Guardar resultado
df_mstl.to_csv(r"E:\SAR_UVa\CSV'S\descomposicion_mstl.csv", index=False)
print("\n✅ Archivo guardado: descomposicion_mstl.csv")

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Cargar datos MSTL
df_mstl = pd.read_csv(r"E:\SAR_UVa\CSV'S\descomposicion_mstl.csv")
df_mstl['fecha'] = pd.to_datetime(df_mstl['fecha'])

# ========== CAMBIAR ESTE VALOR PARA VISUALIZAR OTRA PARCELA ==========
id_parcela_seleccionada = 20
# ===================================================================
for id in df_mstl['ID_PARCELA'].unique()[:10]:
    id_parcela_seleccionada = id
    datos_parcela = df_mstl[df_mstl['ID_PARCELA'] == id_parcela_seleccionada].copy()
    datos_parcela = datos_parcela.sort_values('fecha')

    if len(datos_parcela) == 0:
        print(f"⚠️  No hay datos para ID_PARCELA = {id_parcela_seleccionada}")
        print(f"IDs disponibles: {sorted(df_mstl['ID_PARCELA'].unique())}")
    else:
        n_season = len(seasonal_periods)
        nrows = n_season + 3  # original, trend, estacionales, residual
        fig, axes = plt.subplots(nrows, 1, figsize=(14, 2.5 * nrows))
        
        # Plot 1: Original
        axes[0].plot(datos_parcela['fecha'], datos_parcela['original'], 'b-', linewidth=1.5, label='Original')
        axes[0].set_ylabel('Original', fontsize=10)
        axes[0].set_title(f'MSTL - ID Parcela {id_parcela_seleccionada}', fontsize=12, fontweight='bold')
        axes[0].grid(True, alpha=0.3)
        axes[0].legend(loc='upper right')
        
        # Plot 2: Trend
        axes[1].plot(datos_parcela['fecha'], datos_parcela['trend'], 'g-', linewidth=1.5, label='Trend')
        axes[1].set_ylabel('Trend', fontsize=10)
        axes[1].grid(True, alpha=0.3)
        axes[1].legend(loc='upper right')
        
        # Plots estacionales
        for i, p in enumerate(seasonal_periods):
            col = f'seasonal_{p}'
            axes[2 + i].plot(datos_parcela['fecha'], datos_parcela[col], linewidth=1.5, label=col)
            axes[2 + i].set_ylabel(col, fontsize=10)
            axes[2 + i].grid(True, alpha=0.3)
            axes[2 + i].legend(loc='upper right')
        
        # Plot Residual
        axes[-1].plot(datos_parcela['fecha'], datos_parcela['residual'], 'purple', linewidth=1, label='Residual')
        axes[-1].axhline(y=0, color='k', linestyle='--', alpha=0.3)
        axes[-1].set_xlabel('Fecha', fontsize=10)
        axes[-1].set_ylabel('Residual', fontsize=10)
        axes[-1].grid(True, alpha=0.3)
        axes[-1].legend(loc='upper right')
        
        for ax in axes:
            ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
            plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n📊 Estadísticas para ID_PARCELA = {id_parcela_seleccionada}")
        print(f"Número de observaciones: {len(datos_parcela)}")
        print(f"Rango de fechas: {datos_parcela['fecha'].min()} a {datos_parcela['fecha'].max()}")
        print("\nValores promedio:")
        print(f"  Original: {datos_parcela['original'].mean():.4f}")
        print(f"  Trend: {datos_parcela['trend'].mean():.4f}")
        for p in seasonal_periods:
            print(f"  seasonal_{p}: {datos_parcela[f'seasonal_{p}'].mean():.4f}")
        print(f"  Residual: {datos_parcela['residual'].mean():.4f}")

In [ ]:
from scipy.signal import welch
import numpy as np
import matplotlib.pyplot as plt

# Usar resultados MSTL
df_mstl = pd.read_csv(r"E:\SAR_UVa\CSV'S\descomposicion_mstl.csv")
df_mstl['fecha'] = pd.to_datetime(df_mstl['fecha'])

# Frecuencia de muestreo: 1 observación por semana
fs = 1.0

# Componentes estacionales disponibles
seasonal_cols = [col for col in df_mstl.columns if col.startswith('seasonal_')]

# Iterar por componente estacional
for col in seasonal_cols:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle(f'Análisis de Serie Temporal – {col}', fontsize=16, fontweight='bold', y=0.995)
    
    # Graficar algunas parcelas para no saturar
    for id_parcela in df_mstl['ID_PARCELA'].unique()[:10]:
        ts = (
            df_mstl[df_mstl['ID_PARCELA'] == id_parcela]
            .groupby('fecha')[col]
            .mean()
            .dropna()
            .sort_index()
        )
        
        if len(ts) < 5:
            continue
        
        # Subplot 1: Serie temporal
        axes[0].plot(ts.index, ts.values, linewidth=1.5, alpha=0.7, label=f'ID {id_parcela}')
        
        # Subplot 2: ACF
        acf_values = np.correlate(ts.values - ts.values.mean(),
                                 ts.values - ts.values.mean(), mode='full')
        acf_values = acf_values[len(acf_values)//2:]
        acf_values = acf_values / acf_values[0]
        lags = range(min(100, len(ts)//2))
        axes[1].plot(lags, acf_values[:len(lags)], alpha=0.7, linewidth=1.5,
                    markersize=3, label=f'ID {id_parcela}')
        
        # Subplot 3: Periodograma
        freqs, power = welch(ts.values, fs=fs, detrend='constant')
        mask = freqs > 0
        periods = 1.0 / freqs[mask]
        power = power[mask]
        axes[2].plot(periods, power, linewidth=1.5, alpha=0.7,
                    markersize=4, label=f'ID {id_parcela}')

    # Configurar ejes y leyendas
    axes[0].set_title('Serie Temporal', fontsize=13, fontweight='bold')
    axes[0].legend(ncol=2, fontsize=8)
    axes[0].grid(True, alpha=0.3)
    
    axes[1].set_title('Función de Autocorrelación (ACF)', fontsize=13, fontweight='bold')
    axes[1].set_xlabel('Lag (semanas)', fontsize=11)
    axes[1].set_ylabel('Autocorrelación', fontsize=11)
    axes[1].axvline(52, color='red', linestyle='--', alpha=0.5, linewidth=1.5,
                   label='Lag anual (52 sem)')
    axes[1].axhline(0, color='black', linestyle='-', linewidth=0.8)
    axes[1].legend(ncol=2, fontsize=8, loc='best')
    axes[1].grid(True, alpha=0.3)
    
    axes[2].set_title('Periodograma', fontsize=13, fontweight='bold')
    axes[2].axvline(52, color='red', linestyle='--', alpha=0.6)
    axes[2].set_xlim(0, 300)
    axes[2].legend(ncol=2, fontsize=8)
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()